In [1]:
import pandas as pd

In [2]:
reviews = pd.read_csv('data/winemag-data-130k-v2.csv')

In [ ]:
reviews.head()

In [29]:
wines = reviews[['title','description']]
titles = reviews[['title']]

In [ ]:
# wines = wines.assign(description=wines.description.str.replace('“', ''))
# wines[wines.description.str.contains('“')]

In [ ]:
# What funky characters need to be replaced?
# test = wines[['description']].assign(matched=wines.description.str.extract('([^ ,;:/%&\+\$\(\)\.\?\-–—!…\w\d\'])'))
# test = test[test.matched.notna()]
# test['matched'].value_counts()

In [ ]:
# test[test.matched=='+']['description'][2587]


In [30]:
# Clean descriptions - replace curly punctuation + dashes
wines = wines.assign(clean_desc=wines.description.str.replace('“|”', '"'))
wines = wines.assign(clean_desc=wines.clean_desc.str.replace('’|‘|`|´', '\''))
wines = wines.assign(clean_desc=wines.clean_desc.str.replace('[\-–—]', '- '))

In [31]:
# Clean descriptions - remove certain punctuation
wines = wines.assign(clean_desc=wines.clean_desc.str.replace('[^ ,;:/%&\+\$\(\)\.\?\-–—!…\w\d\']', ''))

In [32]:
# Clean descriptions - remove multiple white spaces
wines = wines.assign(clean_desc=wines.clean_desc.str.replace('\s', ' '))

In [33]:
# Clean descriptions - remove multiple punctuation
wines = wines.assign(clean_desc=wines.clean_desc.str.replace('\.{3}', '…'))

In [34]:
# Clean descriptions - remove white spaces around commas and other punctuation
# wines[wines.clean_desc.str.contains('[,;:/%&\+\$\(\)\.\?\-–—!…]')]

In [35]:
wines = wines.assign(words=wines.clean_desc.str.split(pat=' '))

In [36]:
def split_pairs(x):
    """Split a list into a list of 2-tuples."""
    output = []
    for i in range(0, len(x)):
        if i == 0:
            output.append(tuple(['_START', x[i]]))
        output.append(tuple(x[i:i+2]))
    return output

In [37]:
# sample = wines.iloc[0,3]
# [x for x in split_pairs(sample)]

In [38]:
wines = wines.assign(pairs=wines.apply(lambda row: split_pairs(row.words), axis=1))

In [39]:
words = wines['pairs'].apply(pd.Series).stack().to_frame(name='pairs')

In [40]:
punctuation = ['.', '?', '!']
words = words.assign(
    word_1=words['pairs'].apply(lambda x: '_START' if any([x[0].endswith(p) for p in punctuation]) else x[0]),
    word_2=words['pairs'].apply(lambda x: x[1] if len(x) > 1 else '_END')
)

In [41]:
words.head()

pairs    word_1    word_2
0 0     (_START, Aromas)    _START    Aromas
  1    (Aromas, include)    Aromas   include
  2  (include, tropical)   include  tropical
  3   (tropical, fruit,)  tropical    fruit,
  4     (fruit,, broom,)    fruit,    broom,

In [42]:
words = words.reset_index()
words = words[['pairs', 'word_1', 'word_2']]

In [45]:
# words[['word_1']].groupby('word_1').agg({'word_1': 'count'}).sort_values('word_1')
# words[['word_2']].groupby('word_2').agg({'word_2': 'count'}).sort_values('word_2')

In [46]:
# Remove any empty words, filter out pairs that occur infrequently to make it faster
# len(wordcounts.index)
words = words[(words.word_1 != '') & (words.word_2 != '') & (words.word_2 != '_END')]
# wordcounts = wordcounts[wordcounts.n > 5]

In [ ]:
wsample = words.sample(200)

In [47]:
w1 = words.groupby('word_1').agg('size').reset_index(name='w1_freq')
w2 = words.groupby(['word_1', 'word_2']).agg('size').reset_index(name='n')
wordcounts = w2.merge(w1, on='word_1', how='inner')

In [48]:
wordcounts = wordcounts.assign(weight=wordcounts.n/wordcounts.w1_freq)

In [50]:
wordcounts[wordcounts.word_1 == '_START'].sample(1, weights=wordcounts.weight).iloc[0,1]

'It'

In [63]:
def write_review():
    punctuation = ['.', '!', '?']
    prev_word = '_START'
    review = []
    keep_speaking = True
    while(keep_speaking):
        prev_word = wordcounts[wordcounts.word_1 == prev_word] \
            .sample(1, weights=wordcounts.weight) \
            .iloc[0, 1]
        review.append(prev_word)

        # if it's the end of a sentence, start over
        if any([prev_word.endswith(x) for x in punctuation]):
            prev_word = '_START'

        # if we've generated enough text, take a break
        # pattern = '|'.join(punctuation).replace('.', '\.').replace('?', '\?')
        # num_sentences = len(re.findall(pattern, review))
        num_sentences = sum([w.endswith(p) for w in review for p in punctuation])
        if len(review) >= 40 and num_sentences >= 2 and prev_word == '_START':
            keep_speaking = False

    return ' '.join(review)

# prev_word = '_START'
# review = []
# for i in range(20):
#     prev_word = wordcounts[wordcounts.word_1 == prev_word].sample(1, weights=wordcounts.weight).iloc[0, 1]
#     print(prev_word)
#     if any([prev_word.endswith(x) for x in punctuation]):
#         prev_word = '_START'

In [67]:
write_review()

'Drink from some intensity develops an intensely mineral notes to medium acidity show through, balancing acidity. Fine, dusty and the next 10 years to produce a wine is lower slopes of espresso beans and beautiful. This is a backbone and caramelized sugar.'

In [ ]:
# Titles

In [ ]:
# Clean titles - remove everything in parentheses
titles = titles.assign(clean_title=titles.title.str.replace('\(.*\)', ''))
titles = titles.assign(clean_title=titles.clean_title.str.strip())

In [ ]:
titles = titles.assign(words=titles.clean_title.str.split(pat=' '))

In [ ]:
titles = titles.assign(pairs=titles.apply(lambda row: split_pairs(row.words), axis=1))

In [ ]:
title_words = titles['pairs'].apply(pd.Series).stack().to_frame(name='pairs')

In [ ]:
title_words = title_words.reset_index()
punctuation = ['.', '?', '!']
title_words = title_words.assign(
    word_1=title_words['pairs'].apply(lambda x: '_START' if any([x[0].endswith(p) for p in punctuation]) else x[0]),
    word_2=title_words['pairs'].apply(lambda x: x[1] if len(x) > 1 else '_END')
)
title_words = title_words[['word_1', 'word_2', 'pairs']]

In [ ]:
title_words = title_words[(title_words.word_1 != '') & (title_words.word_2 != '') & (title_words.word_2 != '_END')]

In [ ]:
title_words.head()

In [ ]:
w1 = title_words.groupby('word_1').agg('size').reset_index(name='w1_freq')
w2 = title_words.groupby(['word_1', 'word_2']).agg('size').reset_index(name='n')
title_wordcounts = w2.merge(w1, on='word_1', how='inner')

In [ ]:
title_wordcounts.head()

In [ ]:
title_wordcounts = title_wordcounts.assign(weight=title_wordcounts.n/title_wordcounts.w1_freq)

In [ ]:
prev_word = '_START'
title = []
for i in range(6):
    prev_word = title_wordcounts[title_wordcounts.word_1 == prev_word].sample(1, weights=title_wordcounts.weight).iloc[0, 1]
    print(prev_word)

In [ ]:
len(title_wordcounts.index)

In [ ]:
####### OLD #######

In [ ]:
# Create a matrix of weights
# M = wordcounts[['word_1', 'word_2', 'weight']].pivot(index='word_2', columns='word_1', values='weight')
# M = M.fillna(0)

In [ ]:
M.iloc[:, 0].reset_index()[['word_2']].sample(1, weights=M['_START'])

In [ ]:
wordcounts.sort_values('n')

In [ ]:
# Everything below is not for the matrix approach

In [ ]:
wordcounts = wordcounts.sort_values('count', ascending=False)

In [ ]:
# Group by word_1, create a list of (word_2, count) tuples ordered by count
wordcounts = wordcounts.assign(next_words=wordcounts[['word_2', 'count']].values.tolist())

In [ ]:
starting_words = wordcounts[['word_1', 'next_words']].groupby('word_1').agg(lambda x: list(x))

In [ ]:
starting_words